# Create training dataset

In [332]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from statistics import multimode
from cmath import nan
pd.set_option("display.max_rows", None, "display.max_columns", None)



# Functions

In [333]:
def split_string(string):
    if string == string:
        string = string.replace("'", "")
        out = string.strip('][').split(', ')
        return out
    else:
        return nan

In [334]:
def get_subregion(dist1, dist2, place):
    row = dist1.loc[dist1['Place0']==place]
    shape = row['geometry'].to_list()
    shape = shape[0]

    places = dist2['geometry']
    res = places.within(shape)
    res = res.to_list()
    temp = dist2
    temp['res'] = res
    temp = temp.loc[temp['res']==True]
    return(temp)

### tweets

In [335]:
def plot_Tweets(dist, tweets, n):    
    Nu_Tweets = []
    for index, row in dist.iterrows():
        size = len(tweets.loc[tweets['Place' + str(n)]==row['Place'+ str(n)]])

        Nu_Tweets.append(size)

    Nu_Tweets
    dist['Tweets'] = Nu_Tweets

    plot = dist.explore(
        column="Tweets", # make choropleth based on "Data" column
        tooltip=['Place'+ str(n),'Tweets'], # show "Province, Data" value in tooltip (on hover)
        popup=True, # show all values in popup (on click)
        tiles="CartoDB positron", # use "CartoDB positron" tiles
        cmap="Paired", # https://matplotlib.org/stable/tutorials/colors/colormaps.html
        style_kwds=dict(color="black") # use black outline
        )
    return(plot)

In [336]:
def tweet_rate_av2(dates, tweets):
    tweet_day = [0]*10
    for i, date in enumerate(dates):
        day = date
        d1 = day - timedelta(days=5) 
        d2 = day + timedelta(days=5) 
        delta = (d2-d1).days
        for i in range(delta):
            day = d1+timedelta(days=i)
            NuTweets = len(tweets.loc[tweets['Date']==day]) 
            tweet_day[i]=tweet_day[i]+NuTweets

    # getting the average
    tweet_day =[x/len(dates) for x in tweet_day]

    # normalising
    #tweet_day =[x/max(tweet_day) for x in tweet_day]

    # Getting the average of the sets
    average1 = [sum(tweet_day)/len(tweet_day)]*10

    # this is just for the x axis
    date = list(range(1,10+1))


    plt.plot(date,tweet_day, "-r", label="Tweets")
    plt.plot(date,average1,"-b", label="Average Over All Set Dates")
    plt.axvline(x = 5, color = 'y', label = 'Centre (location of protest/non-protests)')
    plt.legend(loc="upper left")
    plt.ylabel('Number of Tweets')
    plt.xlabel('Date')
    plt.show()

In [337]:
def tweet_rate2(dates, tweets):
    for date in dates:
        day = date
        d1 = day - timedelta(days=5) 
        d2 = day + timedelta(days=5) 
        date2 = []
        tweet_day = []
        delta = (d2-d1).days

        for i in range(delta):
            day = d1+timedelta(days=i)
            NuTweets = len(tweets.loc[tweets['Date']==day]) 
            date2.append(i)
            tweet_day.append(NuTweets)

        # Normalising
        tweet_day =[x/max(tweet_day) for x in tweet_day]
        plt.plot(date2,tweet_day)
        plt.ylabel('Number of Tweets')
        plt.xlabel('Date') 

### Dates

In [338]:
def get_dates(protests,tweets):
    start = tweets.tail(1)['Date'].to_list()
    start = start[0]
    end = tweets.head(1)['Date'].to_list()
    end = end[0]

    delta = (end-start).days
    delta = delta
    yes = []
    no = []
    protests2 = []

    for i in range(delta):
        date = start + timedelta(days=i)
        temp = protests.loc[protests['Date']==date]
        if temp.empty:
            no.append(date)
        else:
            yes.append(date)
    return(yes, no)

### Training set

In [339]:
def get_start(protests, tweets):
    dates = get_dates(protests, tweets)
    yes = dates[0]
    no = dates[1]


    res = [1]*len(yes)
    data = {'Protest':res,'Date':yes}
    yes = pd.DataFrame(data)

    res = [0]*len(no)
    data = {'Protest':res,'Date':no}
    no = pd.DataFrame(data)

    # randomly sampling len(yes) samples from no such that there is an equal number of 
    # protests to non protest
    x = 46 # 0.83 & 0.75
    # x = 54 # 0.87 0.53


    no = no.sample(len(yes), random_state = x)

    # Joining the dataframes, resetting the index and randomising the rows. 
    data = pd.concat([yes,no])
    data = data.sample(frac=1, random_state = x)
    data = data.reset_index(drop=True)

    return(data)

In [340]:
# return(tweet_day,average,specifiedTweets,length)
# return(likes,followers,retweets,replies,Subjectivity,Polarity)
# return(centre)

In [341]:
def get_tweets(date, tweets):
    day = date
    d1 = day - timedelta(days=5) 
    d2 = day + timedelta(days=5) 
    delta = (d2-d1).days
    delta = int(delta)
    tweet_day = [0]*(int(5*2))
    for i in range(10):
        day = d1+timedelta(days=i)
        NuTweets = len(tweets.loc[tweets['Date']==day]) 
        tweet_day[i]=tweet_day[i]+NuTweets
    # average 
    average = sum(tweet_day)/len(tweet_day)
    # normalising
    tweet_day =[x/max(tweet_day) for x in tweet_day]
    # # average 
    # average = sum(tweet_day)/len(tweet_day)
    # specific tweets.
    specifiedTweets = tweets[((tweets['Date'] >d1)&(tweets['Date'] <d2))]
    specifiedTweets = specifiedTweets.reset_index(drop=True)
    length = len(specifiedTweets)

    return(tweet_day,average,specifiedTweets,length)

In [342]:
def concat_lists_to_list(column_name, df):
    content_list  = []
    for iter_t, tweet in df.iterrows():
        if tweet[column_name] == tweet[column_name]:
            for content in tweet[column_name]:
                content_list.append(content)
    return content_list

In [343]:
def tweet_metrics(tweets):
    length = len(tweets)
    if length == 0:
        length = 1
    followers = (tweets['author_followers'].sum())/length
    retweets = (tweets['retweets'].sum())/length
    replies = (tweets['replies'].sum())/length
    likes = (tweets['likes'].sum())/length
    Subjectivity = (tweets['Subjectivity'].sum())/length
    Polarity = (tweets['Polarity'].sum())/length
    

    grievance_list = concat_lists_to_list('grievances', tweets)
    greivance_mode = multimode(grievance_list)

    triggers_list = concat_lists_to_list('triggers', tweets)
    triggers_mode = multimode(triggers_list)

    tactics_list = concat_lists_to_list('tactics', tweets)
    tactics_mode = multimode(tactics_list)

    actors_list = concat_lists_to_list('actors', tweets)
    actors_mode = multimode(actors_list)

    locations_list = concat_lists_to_list('locations', tweets)
    locations_mode = multimode(locations_list)

    weapons_list = concat_lists_to_list('weapons', tweets)
    weapons_mode = multimode(weapons_list)

    eventualities_list = concat_lists_to_list('eventualities', tweets)
    eventualities_mode = multimode(eventualities_list)

    curiosities_list = concat_lists_to_list('curiosities', tweets)
    curiosities_mode = multimode(curiosities_list)

    non_protests_list = concat_lists_to_list('non_protests', tweets)
    non_protests_mode = multimode(non_protests_list)

    universities_list = concat_lists_to_list('universities', tweets)
    universities_mode = multimode(universities_list)


    return(likes,followers,retweets,replies,Subjectivity,Polarity,greivance_mode,triggers_mode,
            tactics_mode,actors_mode,locations_mode,weapons_mode,eventualities_mode,
            curiosities_mode,non_protests_mode,universities_mode)

In [344]:
def get_centre(tweets):
    # locations = tweets['geometry'].centroid
    # print(locations.head())
    # locations = locations.reset_index(drop=True)
    # centre = locations.centriod

    # its not working for now
    # centre = 'too bad'
    # return(centre)
    

    # We are going to return the place
    place = tweets['Place1'].value_counts().idxmax()
    return(place)
    

# Training Data

In [345]:
def get_training(data, tweets):
    NuTweets = []
    average = []
    d1 = []
    d2 = []
    d3 = []
    d4 = []
    d5 = []
    d6 = []
    d7 = []
    d8 = []
    d9 = []
    d10 = []

    likes = []
    followers = []
    retweets = []
    replies = []
    sub = []
    pol = []
    griev = []
    trigg = []
    tact = []
    act = []
    loca = []
    weap = []
    even = []
    curi = []
    nonp = []
    uni = []

    centre = []


    for index, row in data.iterrows():
        date = row['Date']
        
        res1 = get_tweets(date,tweets)
        res2 = tweet_metrics(res1[2])
        res3 = get_centre(res1[2])


        average.append(res1[1])
        NuTweets.append(res1[3])
        d1.append(res1[0][0])
        d2.append(res1[0][1])
        d3.append(res1[0][2])
        d4.append(res1[0][3])
        d5.append(res1[0][4])
        d6.append(res1[0][5])
        d7.append(res1[0][6])
        d8.append(res1[0][7])
        d9.append(res1[0][8])
        d10.append(res1[0][9])


        likes.append(res2[0])
        followers.append(res2[1])
        retweets.append(res2[2])
        replies.append(res2[3])
        sub.append(res2[4])
        pol.append(res2[5])
        griev.append(res2[6])
        trigg.append(res2[7])
        tact.append(res2[8])
        act.append(res2[9])
        loca.append(res2[10])
        weap.append(res2[11])
        even.append(res2[12])
        curi.append(res2[13])
        nonp.append(res2[14])
        uni.append(res2[15])
        

        centre.append(res3)

    

    metrics = {'NuTweets':NuTweets,'d1':d1,'d2':d2,'d3':d3,'d4':d4,
               'd5':d5,'d6':d6,'d7':d7,'d8':d8,'d9':d9,'d10':d10,
               'average':average,'likes':likes,'followers':followers,
               'retweets':retweets,'replies':replies,'sub':sub,'pol':pol,
               'place':centre,'grievances':griev, 'triggers': trigg,
               'tactics': tact, 'actors': act, 'locations': loca,
               'weapons': weap, 'eventualities': even, 'curiosities': curi,
               'non_protests':nonp, 'universities':uni}
    metrics = pd.DataFrame(metrics)

        
    training = pd.concat([data,metrics.reindex(data.index)], axis=1)
    training = training.drop(['Date'], axis = 1)
    return(training)

# Models

In [346]:
def logistic_regression(training):
    X_train,X_test,y_train,y_test = train_test_split(training.drop(['Protest','place', 'grievances', 'triggers', 'tactics', 'actors', 'locations', 'weapons', 'eventualities', 'curiosities', 'non_protests', 'universities'],axis = 1), training['Protest'],test_size = 0.3,random_state=1)
    # training the model
    logreg =  LogisticRegression(solver='lbfgs')
    logreg.fit(X_train,y_train)
    score = logreg.score(X_test,y_test)
    return(logreg,score)

In [347]:
def niave(training):
    X_train,X_test,y_train,y_test = train_test_split(training.drop(['Protest','place', 'grievances', 'triggers', 'tactics', 'actors', 'locations', 'weapons', 'eventualities', 'curiosities', 'non_protests', 'universities'],axis = 1), training['Protest'],test_size = 0.2)
    naive = GaussianNB()
    naive.fit(X_train,y_train)
    score = naive.score(X_test,y_test)
    return(naive,score)

In [348]:
def linearSVM(training):
    from sklearn.svm import LinearSVC
    X_train,X_test,y_train,y_test = train_test_split(training.drop(['Protest','place', 'grievances', 'triggers', 'tactics', 'actors', 'locations', 'weapons', 'eventualities', 'curiosities', 'non_protests', 'universities'],axis = 1), training['Protest'],test_size = 0.2)
    svm = LinearSVC()
    svm.fit(X_train,y_train)
    score = svm.score(X_test,y_test)
    return(svm,score)

# MAIN

### Downloading tweets and protests

In [349]:
tweets = pd.read_csv('DATA/tweets.csv')

In [350]:
tweets['grievances'] = tweets['grievances'].apply(lambda x: split_string(x))
tweets['triggers'] = tweets['triggers'].apply(lambda x: split_string(x)) 
tweets['tactics'] = tweets['tactics'].apply(lambda x: split_string(x)) 
tweets['actors'] = tweets['actors'].apply(lambda x: split_string(x)) 
tweets['locations'] = tweets['locations'].apply(lambda x: split_string(x)) 
tweets['weapons'] = tweets['weapons'].apply(lambda x: split_string(x)) 
tweets['eventualities'] = tweets['eventualities'].apply(lambda x: split_string(x)) 
tweets['curiosities'] = tweets['curiosities'].apply(lambda x: split_string(x)) 
tweets['non_protests'] = tweets['non_protests'].apply(lambda x: split_string(x)) 
tweets['universities'] = tweets['universities'].apply(lambda x: split_string(x))

In [351]:

tweets['geometry'] = gpd.GeoSeries.from_wkt(tweets['geometry'])
tweets['Date']=pd.to_datetime(tweets['Date'], format='%Y %m %d')
tweets = tweets.drop(['Unnamed: 0','Analysis'],axis=1)

In [352]:
protests = pd.read_csv('DATA/protests.csv')
protests['geometry'] = gpd.GeoSeries.from_wkt(protests['geometry'])
protests['Date']=pd.to_datetime(protests['Date'], format='%Y %m %d')
protests = protests.drop(['Unnamed: 0'],axis=1)
protests.drop_duplicates(['Date','Place0'],keep= 'first',inplace =True)
print(len(protests))

99


In [353]:
tweets.head(1)

,text,Date,geometry,author_followers,retweets,replies,likes,quote_count,Subjectivity,Polarity,grievances,triggers,tactics,actors,locations,weapons,eventualities,curiosities,non_protests,universities,Place0,Place1,Place2,Place3
0,#KZN We are on thee way! Hitting up Richards B...,2017-12-31,POINT (28.23140 -25.75450),437807,0,1,21,0,0.066667,0.0,NaN,NaN,[Gathering],[Political Party],NaN,NaN,NaN,NaN,NaN,[University of Pretoria],Gauteng,City of Tshwane,City of Tshwane,56


In [354]:
protests.head(1)

,Date,notes,geometry,Place0,Place1,Place2,Place3
0,2017-10-25,A few hundred Fees Must Fall protesters attemp...,POINT (18.47640 -33.96330),Western Cape,City of Cape Town,City of Cape Town,58


# Lets test protest in a chosen place

In [355]:
places = protests['Place0'].unique()
print(places)

['Western Cape' 'KwaZulu-Natal' 'Gauteng' 'Eastern Cape' 'Free State'
 'North West' 'Limpopo']


In [356]:
print(tweets['Place0'].unique())

['Gauteng' 'KwaZulu-Natal' 'Western Cape' 'Free State' 'Eastern Cape'
 'Mpumalanga' 'North West' 'Limpopo' 'Nothern Cape' '0']


In [357]:
sa_training = pd.DataFrame()
for place in places:
    p = protests.loc[(protests['Place0'] == place)]
    t = tweets.loc[(tweets['Place0'] == place)]
    if p.empty == False:
        data = get_start(p, t)
        training = get_training(data, t)
        sa_training = pd.concat([sa_training, training])
sa_training.reset_index(drop = True, inplace = True)


In [358]:
# Expected Time for 20k tweets = seconds
# data = get_start(protests,tweets)
# training = get_training(data,tweets)
# training.head(5)

In [359]:
sa_training.to_csv('DATA/training_data.csv')

In [360]:
sa_training.shape

(198, 30)

# Creating a simple model

In [361]:
log = logistic_regression(sa_training)
log[1]

0.8333333333333334

In [362]:
log = niave(training)
log[1]

c:\Users\papag\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\naive_bayes.py:487: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
c:\Users\papag\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\naive_bayes.py:488: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
c:\Users\papag\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\naive_bayes.py:488: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


0.0

In [363]:
log = linearSVM(sa_training)
log[1]

c:\Users\papag\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.7

In [364]:
sa_training.head()

,Protest,NuTweets,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,average,likes,followers,retweets,replies,sub,pol,place,grievances,triggers,tactics,actors,locations,weapons,eventualities,curiosities,non_protests,universities
0,1,156,0.409091,0.181818,0.068182,0.068182,0.500000,1.000000,0.500000,0.545455,0.454545,0.227273,17.4,0.801282,11181.128205,0.826923,0.262821,0.468301,0.092900,City of Cape Town,[Education],[Court hearing],[Gathering],[Political Party],[Tertiary Edu],"[Police weapons, Crowd projectiles, Crowd weap...",[Police attack],[Special Keywords],"[Football matches, Other sport, Election campa...",[Stellenbosh University]
1,0,38,0.222222,0.555556,0.444444,0.444444,0.444444,0.111111,0.444444,0.111111,1.000000,0.666667,4.0,1.078947,31909.263158,0.763158,0.236842,0.297430,0.090393,City of Cape Town,[Education],[Wage disputes],[Gathering],[Political Party],[Tertiary Edu],[],[],[],"[Election campaigns, Exhibitions, Other sport]",[Stellenbosh University]
2,1,128,0.176471,0.941176,1.000000,0.823529,0.823529,0.882353,0.588235,0.647059,0.823529,1.000000,13.1,1.054688,17116.125000,1.828125,0.304688,0.459057,0.100919,City of Cape Town,[Education],[Project begin],"[Disrupt, Gathering]",[Political Party],[Tertiary Edu],[Police weapons],[Police attack],[Special Keywords],[Election campaigns],[Stellenbosh University]
3,0,40,0.222222,0.444444,0.555556,0.666667,0.222222,0.111111,0.222222,1.000000,0.444444,0.777778,4.2,1.450000,8602.425000,1.075000,0.425000,0.293188,0.086130,Cape Winelands,[Education],[Court hearing],[Gathering],[Political Party],[School],[Crowd projectiles],[],[Special Keywords],"[Election campaigns, 16 Days of activism]",[Stellenbosh University]
4,1,164,0.204545,0.409091,0.181818,0.068182,0.068182,0.500000,1.000000,0.500000,0.545455,0.454545,17.3,0.768293,11386.274390,0.817073,0.262195,0.464095,0.089726,City of Cape Town,[Education],[Court hearing],[Gathering],[Political Party],[Tertiary Edu],"[Crowd projectiles, Crowd weapons]",[Police attack],[Special Keywords],"[Football matches, Other sport, Election campa...",[Stellenbosh University]
